In [264]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import time
import multiprocessing as mp
import sys
import dataAck
import curveTreeDB


In [314]:
def getPertinentDataModels(db):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=db)
#             query.add_filter("numberOfPredictors", "=", 2)
            query.add_filter("IS_PROFITABILITY", '>', 0.5) #-0.05
            retrievedModels = list(query.fetch())
            return retrievedModels
        except:
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))
            return 

In [348]:
allValidModels = getPertinentDataModels(params.curveModels)

In [349]:
quickModels = []
predLength = {}
for mod in allValidModels:
#     if mod["predictionLength"] < 10:
#         continue
    if mod["predictionLength"] not in predLength:
        predLength[mod["predictionLength"]] = []
    predLength[mod["predictionLength"]].append(abs(mod["OOS_BETA"] - mod["IS_BETA"]))
#     print(mod["IS_BETA"], mod["OOS_BETA"], mod["predictionLength"])
    quickModels.append(mod)

# for item in sorted(predLength):
#     print(item, sum(predLength[item])/len(predLength[item]))

In [350]:
len(quickModels)

420

In [353]:
import pandas as pd
cleanedModels = []
seenTickers = {}
seenPredictors = {}
seenAlpha = {}
tickerSharpe = {}

for item in quickModels:


    if item["IS_MIN PROFITABILITY 45"] < 0.3:
        continue

    if item["IS_RAW BETA"] > 0.3:
        continue



    if "series" in item:
        item["DAYS BACK"] = int(item["series"].split(",")[1])
        item["SMOOTHING"] = int(item["series"].split(",")[4][:-1])

    if "numberOfPredictors" not in item:
        item["numberOfPredictors"] = 0

    item["DISTANCE PER POINT"] = item["maxDistance"]/ item["lookbackDistance"]

    if item["numberOfPredictors"] not in seenPredictors:
        seenPredictors[item["numberOfPredictors"]] = {"better":0, "greaterThanZero":0, "bad":0, "total":0}
        seenAlpha[item["numberOfPredictors"]] = {"greaterThanZero":0, "total":0}

    seenPredictors[item["numberOfPredictors"]]["total"] += 1
    seenAlpha[item["numberOfPredictors"]]["total"] += 1
    if item["OOS_PROFITABILITY"] > item["OOS_FACTOR PROFITABILITY"]:
#         print(item["ticker"], item["OOS_SHARPE"], item["OOS_FACTOR SHARPE"] )
        seenPredictors[item["numberOfPredictors"]]["better"] += 1
    if item["OOS_PROFITABILITY"] > 0.5:
        seenPredictors[item["numberOfPredictors"]]["greaterThanZero"] += 1
    if item["OOS_PROFITABILITY"] < 0.5 and item["OOS_FACTOR PROFITABILITY"] > 0.5:
        seenPredictors[item["numberOfPredictors"]]["bad"] += 1
    if item["OOS_ALPHA"] > 0:
        seenAlpha[item["numberOfPredictors"]]["greaterThanZero"] += 1

    print("ticker", item["ticker"])
    print(item["series"])
    print("NUMBER OF PREDICTORS", item["numberOfPredictors"])
    print("SHARPE", item["IS_SHARPE"], item["OOS_SHARPE"], item["OOS_FACTOR SHARPE"])
    print("RAW BETA", item["IS_RAW BETA"], item["OOS_RAW BETA"])
    print("BETA", item["IS_BETA"], item["OOS_BETA"])
    print("ALPHA", item["IS_ALPHA"], item["OOS_ALPHA"])
    print("MIN PROFITABILITY 45", item["IS_25TH PROFITABILITY 45"], item["IS_MIN PROFITABILITY 45"])
    print("MIN PROFITABILITY 90", item["IS_25TH PROFITABILITY 90"], item["IS_MIN PROFITABILITY 90"])
    print("PROFITABILITY", item["IS_PROFITABILITY"], item["OOS_PROFITABILITY"])
    print("PROFITABILITY DIFFERENCE", item["IS_PROFITABILITY DIFFERENCE"])
    print("SHARPE DIFFERENCE", item["IS_SHARPE DIFFERENCE"])
    print("DISTANCE PER POINT", item["lookbackDistance"], item["DISTANCE PER POINT"])
    print("**********")



    if item["ticker"] not in seenTickers:
        seenTickers[item["ticker"]] = 0
    if item["ticker"] not in tickerSharpe:
        tickerSharpe[item["ticker"]] = item["OOS_FACTOR SHARPE"]
    seenTickers[item["ticker"]] += 1
    itemKeys= list(item.keys())
    for key in itemKeys:
        if key.startswith("OOS") and key != "OOS_ALPHA" and key != "OOS_PROFITABILITY DIFFERENCE" and key != "OOS_FACTOR SHARPE" and key != "OOS_FACTOR PROFITABILITY" and key != "OOS_RAW BETA" and key != "OOS_PROFITABILITY" and key != "OOS_SHARPE DIFFERENCE SLIPPAGE" and key != "OOS_SHARPE" and key != "OOS_BETA" and key != "OOS_ANNUALIZED RETURN":
            del item[key]
    cleanedModels.append(item)
print(seenTickers)
print(tickerSharpe)
for predictors in sorted(seenPredictors):
    print(predictors, seenPredictors[predictors]["total"],\
          1.0 - seenPredictors[predictors]["bad"]/seenPredictors[predictors]["total"],\
          seenPredictors[predictors]["better"]/seenPredictors[predictors]["total"],\
          seenPredictors[predictors]["greaterThanZero"]/seenPredictors[predictors]["total"])
    
    print("ALPHA", predictors, seenAlpha[predictors]["greaterThanZero"]/seenAlpha[predictors]["total"])
    
    
    
    
    

ticker EWC
('SDY', 40, None, 12, 3)
NUMBER OF PREDICTORS 0
SHARPE 0.37433999426870407 1.8282596142172982 0.7008396716515909
RAW BETA 0.11837216460876769 0.01569294762715093
BETA 0.1311132550930726 0.0016831752717556068
ALPHA 0.04546110192746984 0.14872288417113524
MIN PROFITABILITY 45 0.4444444444444444 0.3333333333333333
MIN PROFITABILITY 90 0.45555555555555555 0.37777777777777777
PROFITABILITY 0.5002597402597403 0.49603174603174605
PROFITABILITY DIFFERENCE -0.031688311688311654
SHARPE DIFFERENCE -0.08094174553116157
DISTANCE PER POINT 44 0.045455
**********
ticker LQD
('KRE', 17, 21, None, 2)
NUMBER OF PREDICTORS 0
SHARPE 1.290680859047061 0.48667191101974117 0.250020266547937
RAW BETA 0.28576370357192415 0.05755578093306263
BETA 0.18125914581265204 0.052621064164704245
ALPHA 0.03572341175880201 0.013167081061699289
MIN PROFITABILITY 45 0.4444444444444444 0.3333333333333333
MIN PROFITABILITY 90 0.45555555555555555 0.3888888888888889
PROFITABILITY 0.500276090557703 0.5396825396825397


In [352]:
len(cleanedModels)

11

In [343]:
cm = pd.DataFrame(cleanedModels)
for col in cm.columns.values:
    
#     try:
    print(col)
    print("____________________")
    for target in ["OOS_PROFITABILITY", "OOS_SHARPE", "OOS_ALPHA", "OOS_PROFITABILITY DIFFERENCE"]:
        groups = cm.groupby([pd.cut(cm[col].values, 5)])[target]
        print(groups.mean(), groups.size())
    print("**********")
        
#         groupsSharpe = cm.groupby([col])["OOS_PROFITABILITY"].mean()
#         if len(groupsSharpe) < 20:
#             print(groupsSharpe)
#             print(cm.groupby([col])["OOS_SHARPE DIFFERENCE SLIPPAGE"].mean())
#             print(cm.groupby([col])["OOS_SHARPE"].mean())
#             print(cm.groupby([col])["OOS_ALPHA"].mean())
#         print(col, "OOS_PROFITABILITY")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_PROFITABILITY"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
#         print(col, "OOS_SHARPE DIFFERENCE SLIPPAGE")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_SHARPE DIFFERENCE SLIPPAGE"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_SHARPE")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_SHARPE"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_ALPHA")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_ALPHA"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_PROFITABILITY DIFFERENCE")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_PROFITABILITY DIFFERENCE"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_BETA")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_BETA"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_ANNUALIZED RETURN")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_ANNUALIZED RETURN"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_FACTOR SHARPE")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_FACTOR SHARPE"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
#         plt.draw()
#     except:
#         continue
    



DAYS BACK
____________________
(0.957, 9.6]    0.502976
(9.6, 18.2]     0.502232
(18.2, 26.8]    0.493079
(26.8, 35.4]    0.493416
(35.4, 44.0]    0.503268
Name: OOS_PROFITABILITY, dtype: float64 (0.957, 9.6]    48
(9.6, 18.2]     64
(18.2, 26.8]    43
(26.8, 35.4]    44
(35.4, 44.0]    51
Name: OOS_PROFITABILITY, dtype: int64
(0.957, 9.6]   -0.005148
(9.6, 18.2]     0.436028
(18.2, 26.8]    0.304523
(26.8, 35.4]   -0.031981
(35.4, 44.0]    0.253542
Name: OOS_SHARPE, dtype: float64 (0.957, 9.6]    48
(9.6, 18.2]     64
(18.2, 26.8]    43
(26.8, 35.4]    44
(35.4, 44.0]    51
Name: OOS_SHARPE, dtype: int64
(0.957, 9.6]   -0.017326
(9.6, 18.2]     0.028434
(18.2, 26.8]    0.010022
(26.8, 35.4]   -0.017099
(35.4, 44.0]    0.010628
Name: OOS_ALPHA, dtype: float64 (0.957, 9.6]    48
(9.6, 18.2]     64
(18.2, 26.8]    43
(26.8, 35.4]    44
(35.4, 44.0]    51
Name: OOS_ALPHA, dtype: int64
(0.957, 9.6]   -0.031498
(9.6, 18.2]    -0.025050
(18.2, 26.8]   -0.034515
(26.8, 35.4]   -0.031926
(35.4

TypeError: can't concat bytes to float